In [42]:
!pip install yfinance
import yfinance as yf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import datetime
from IPython.core.display import clear_output
clear_output()

In [ ]:
# Set Charting Settings
plt.rcParams["figure.figsize"] = (14,10)
plt.rcParams.update({'font.size': 20})

# **Dividend Discount Model**

**The dividend discount model (DDM) is a quantitative method used for predicting the price of a company's stock based on the theory that its present-day price is worth the sum of all of its future dividend payments when discounted back to their present value.**

## **Gordon Growth Model**

**The Gordon growth model is used to determine the intrinsic value of a stock based on a future series of dividends that grow at a constant rate.**

$\Large{\text{Estimated Stock Value} = \frac{\text{Expected Dividend per Share}}{\text{Cost of Equity}-\text{Dividend Growth Rate}}}$ 

 


$\large{\text{Where: Cost of Equity = Risk Free Rate + β(Market Return-Risk Free Rate)}}\\ \large{\text{Dividend Growth Rate = (Return on Equity)(1-Payout Ratio)}}$

This document describes a set of Python functions used for financial analysis and visualization, primarily based on the Gordon Growth Model (GGM). The functions are designed to analyze stock tickers, calculate various financial metrics, and create interactive charts to visualize the results.

## Functions

### `get_cost_of_equity(ticker)`
- Calculates the cost of equity for a given stock ticker.
- Parameters:
  - `ticker`: Stock ticker symbol.
- Returns:
  - `cost_of_equity`: Cost of equity value.

### `get_div_growth_rate(ticker)`
- Calculates the dividend growth rate for a given stock ticker.
- Parameters:
  - `ticker`: Stock ticker symbol.
- Returns:
  - `div_growth_rate`: Dividend growth rate value.

### `get_div_per_share(ticker)`
- Calculates the dividend per share for a given stock ticker.
- Parameters:
  - `ticker`: Stock ticker symbol.
- Returns:
  - `div_per_share`: Dividend per share value.

### `ggm_model(div_per_share, cost_of_equity, div_growth_rate)`
- Implements the Gordon Growth Model to forecast stock prices.
- Parameters:
  - `div_per_share`: Dividend per share value.
  - `cost_of_equity`: Cost of equity value.
  - `div_growth_rate`: Dividend growth rate value.
- Returns:
  - `ggm_forecasted_price`: Forecasted stock price.

### `analyze_ticker(ticker_symbol)`
- Analyzes a stock ticker by calculating various financial metrics and forecasting stock prices using the GGM.
- Parameters:
  - `ticker_symbol`: Stock ticker symbol.
- Returns:
  - `risk_free_rate`: Risk-free rate value.
  - `div_growth_rate`: Dividend growth rate value.
  - `div_per_share`: Dividend per share value.
  - `ggm_forecasted_price`: Forecasted stock price.
  - `beta`: Beta value.

### `create_interactive_chart(x, y, title, x_label, y_label)`
- Creates an interactive chart to visualize GGM forecasted prices.
- Parameters:
  - `x`: X-axis data.
  - `y`: Y-axis data.
  - `title`: Title of the chart.
  - `x_label`: X-axis label.
  - `y_label`: Y-axis label.

In [11]:
def get_cost_of_equity(ticker):
    beta = ticker.info['beta']
    risk_free_rate = yf.Ticker('^TNX').info['previousClose'] * 0.01
    market_return = 0.11
    return risk_free_rate + beta * (market_return - risk_free_rate)

def get_div_growth_rate(ticker):
    return ticker.info['returnOnEquity'] * (1 - ticker.info['payoutRatio'])

def get_div_per_share(ticker):
    return ticker.info['previousClose'] * ticker.info['dividendYield']

def ggm_model(div_per_share, cost_of_equity, div_growth_rate):
    return div_per_share / (cost_of_equity - div_growth_rate)

def analyze_ticker(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    
    try:
        beta = ticker.info['beta']
        if beta is None:  # Check if beta is None and set a default value
            beta = 1  # Default beta value if not provided
    except KeyError:
        print(f"Beta value for {ticker_symbol} is not available. Using default beta of 1.")
        beta = 1  # Default beta value if not provided
    
    risk_free_rate = yf.Ticker('^TNX').info['previousClose'] * 0.01
    market_return = 0.11
    
    try:
        div_growth_rate = ticker.info['returnOnEquity'] * (1 - ticker.info['payoutRatio'])
        div_per_share = ticker.info['previousClose'] * ticker.info['dividendYield']
        ggm_forecasted_price = ggm_model(div_per_share, risk_free_rate + beta * (market_return - risk_free_rate), div_growth_rate)
    except KeyError as e:
        print(f"Required financial data not available for {ticker_symbol}: {e}")
        return None, None, None, None  # Return None values if data is not available
    
    return risk_free_rate + beta * (market_return - risk_free_rate), div_growth_rate, div_per_share, ggm_forecasted_price, beta  # Return beta as well


def create_interactive_chart(x, y, title, x_label, y_label):
    fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines+markers'))
    fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label)
    fig.show()

# Example usage for 'SWK'
ticker_symbol = 'SWK'
results = analyze_ticker(ticker_symbol)  # Use results directly, instead of unpacking
if None not in results:
    cost_of_equity, div_growth_rate, div_per_share, ggm_forecasted_price, beta = results
    print(f'The Gordon Growth Model forecasts the {ticker_symbol} stock price at: ${ggm_forecasted_price:.2f}')

    # Correction for the beta not defined error
    market_return_array = np.arange(0.1, 0.25, 0.01)
    risk_free_rate = yf.Ticker('^TNX').info['previousClose'] * 0.01  # This should be refetched or passed if it changes
    ggm_mr_estimates = [ggm_model(div_per_share, risk_free_rate + beta * (mr - risk_free_rate), div_growth_rate) for mr in market_return_array]

    # Assume create_interactive_chart is defined as before
    create_interactive_chart(market_return_array, ggm_mr_estimates, 'Gordon Growth Dividend Discount Model', 'Required Market Return', 'Forecasted Stock Price')
else:
    print(f"Skipping {ticker_symbol} due to incomplete data.")

# Example usage for a list of assets or Portfolio
assets = ['JEPI', 'AGNC', 'O', 'NUSI', 'CSHI']

# Dictionary to hold the analysis results for each asset
analysis_results = {}

# Analyze each asset and store the results
for asset in assets:
    results = analyze_ticker(asset)
    if None not in results:
        analysis_results[asset] = results
        print(f'The Gordon Growth Model forecasts the {asset} stock price at: ${results[3]:.2f}')
    else:
        print(f"Skipping {asset} due to incomplete data.")

The Gordon Growth Model forecasts the SWK stock price at: $44.86


Beta value for JEPI is not available. Using default beta of 1.
Required financial data not available for JEPI: 'returnOnEquity'
Skipping JEPI due to incomplete data.
The Gordon Growth Model forecasts the AGNC stock price at: $2.14
The Gordon Growth Model forecasts the O stock price at: $21.19
Beta value for NUSI is not available. Using default beta of 1.
Required financial data not available for NUSI: 'returnOnEquity'
Skipping NUSI due to incomplete data.
Beta value for CSHI is not available. Using default beta of 1.
Required financial data not available for CSHI: 'returnOnEquity'
Skipping CSHI due to incomplete data.


## **Multistage Dividend Discount Model**

